In [350]:
import numpy;
import math;
import torch;
import matplotlib.pyplot as plt

plot = False

def getData(N, variance):
    """
    Generates a dataset {(xi, yi) : i = 1, 2, . . .N} of N (X, Y ) pairs for a given value of N and sigma^2
    :return: The generated dataset of (X,Y) pairs
    """
    X = numpy.random.rand(N) # N values between 0 and 1
    Z = numpy.random.normal(0, math.sqrt(variance), N) # Zero mean Gaussian random variable, with standard deviation, of length N
    Y = numpy.cos(2*math.pi*X) + Z # Y generated from function (1)
    #print("X: ", X)
    #print("Z: ", Z)
    #print ("Y: ", Y)

    # store the (X,Y) pairs in a N x 2 matrix
    x_y_pair_matrix = numpy.column_stack((X, Y))
    #print("(X,Y) pairs:", x_y_pair_matrix)
    # optionally plot the dataset that is created
    if plot:
        plt.scatter(X,Y, label="My Random Dataset", marker='x')
        plt.title('Generated Dataset for cos(2*pi*x) from 0 to 1')
    return x_y_pair_matrix

def getMSE_Updated(data_set, poly_coefficients):
    x_values = data_set[:, 0] # Values of x from given dataset
    y_true_values = data_set[:, 1] # Values of y from given dataset

    y_predicted= numpy.dot(poly_coefficients, x_values**numpy.arrange(len(poly_coefficients)))

    # MSE calculation using numpy, but will manually calculate below
    mse = numpy.mean((y_true_values - y_predicted)**2) 
    print ("Automatic MSE:", mse)

    # Sum all the squared differences of each y_pred, y_true. Then average that over the vector length, N 
    summation_diffs_squared = 0
    for y_pred_i, y_true_i in zip(y_predicted, y_true_values):
        squared_diff = (y_pred_i - y_true_i) ** 2
        summation_diffs_squared += squared_diff
    mse = summation_diffs_squared / len(y_true_values)
    print ("Calculated MSE: ", mse)
    return mse

# Old MSE algorithm, I think it took the wrong values (y_pred, y_true isn't enough)
def getMSEOld(y_predicted, y_true): # this is probably wrong, likely needs to take a dataset (X,Y) and a polynomial functions (x^2 + x + 2 ...)
    """
    Get the Mean Square Error (MSE) loss from the predicted dataset, and the true dataset
    :return: The Mean Square Error (MSE)
    """
    # MSE calculation using numpy, but will manually calculate below
    mse = numpy.mean((y_true - y_predicted)**2) 
    #print ("Automatic MSE:", mse)

    # print("\n")
    # print(y_predicted)
    # print(y_true)
    # print("\n")

    # N = len(y_predicted) # This one needs basic arrays, arrays in arrays fail
    # summation_diffs_squared = 0
    # for i in range(len(y_predicted)):
    #     squared_diff = (y_predicted[i] - y_true[i]) ** 2
    #     summation_diffs_squared += squared_diff
    # mse = summation_diffs_squared / N
    # print ("Calculated MSE: ", mse)
    return mse

def fitData(data_set, degree_poly):
    """
    fitData method that estimates the polynomial coefficients by fitting a given data to a degree-d polynomial
    :return: estimated_coefficients, Ein (MSE), Eout
    """
    # Hyperparameters of GD
    learning_rate = 0.1
    iterations = 400
    X_values = numpy.array(data_set[:, 0]) # Values of x from given dataset, vector is 1xN
    Y_values = numpy.array(data_set[:, 1]) # Values of y from given dataset, vector is 1xN
    N = len(Y_values)
    print("Real X values:\n", X_values)
    print("Real Y values:\n", Y_values)

    # Random starting coefficents
    weights = startingRandomCoeffs(degree_poly+1) # vector is 1xD
    print("Staring Weights:\n", weights)

    X_polynomial = numpy.poly1d(weights)
    print("Poly function auto\n", X_polynomial, "\n")

    # Vandermonde matrix, for all X values, without coefficients of N X (D+1) size
    vander_matrix = numpy.vander(X_values, degree_poly+1, True)
    print("Vander Matrix:\n", vander_matrix)

    for x in range(iterations):

        Y_pred = Y_values - vander_matrix.dot(weights)
        
        grad = 2*(learning_rate/N) * ((Y_values - Y_pred)).dot(vander_matrix)
        if x % 100 is 0:
            #print("Predicted Y values:\n", Y_pred)
            print("Current MSE:", getMSEOld(Y_pred, Y_values))
            print("Gradients:\n", grad)
        # weights_new = weights_old + 
        weights -= grad

    if plot:
        plt.scatter(X_values,Y_pred, label="My Random Dataset", marker='x')
        plt.title('Predicted Y Values')

    Ein = 2.345
    Eout = 4.567
    return weights, Ein, Eout # this should be changed to the eventual estimated coefficients

def startingRandomCoeffs(d):
    """
    Get starting random coefficients, a0, a1, ..., ad, for the polynomial regression function
    :return: Randome coefficients for the polynomial based on random values between 0 and 1
    """
    return numpy.random.rand(d)

<>:103: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:103: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Nathan\AppData\Local\Temp\ipykernel_16008\2568880929.py:103: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if x % 100 is 0:


In [351]:

N = 15        # Size of the training dataset
variance = 0.003  # Variance, sigma^2
degree_d=3      # Degree of the model polynomial


data_set = getData(N, variance)
M = 35      # Number of trials
fitData(data_set, degree_d)


Real X values:
 [0.91273511 0.61939943 0.25736519 0.13222844 0.74541659 0.91643526
 0.81031938 0.73777098 0.37120381 0.72349531 0.28427515 0.11697829
 0.51219029 0.79351695 0.77130879]
Real Y values:
 [ 0.97479068 -0.68104601 -0.0078776   0.69938213  0.00302252  0.88089714
  0.3540664  -0.13690844 -0.73474404 -0.24915848 -0.20834948  0.68580854
 -0.90782718  0.27828644  0.13347871]
Staring Weights:
 [0.81576401 0.95492212 0.66910576 0.63788053]
Poly function auto
         3          2
0.8158 x + 0.9549 x + 0.6691 x + 0.6379 

Vander Matrix:
 [[1.         0.91273511 0.83308537 0.76038627]
 [1.         0.61939943 0.38365565 0.23763609]
 [1.         0.25736519 0.06623684 0.01704706]
 [1.         0.13222844 0.01748436 0.00231193]
 [1.         0.74541659 0.55564589 0.41418767]
 [1.         0.91643526 0.83985359 0.76967145]
 [1.         0.81031938 0.6566175  0.53206989]
 [1.         0.73777098 0.54430602 0.40157319]
 [1.         0.37120381 0.13779227 0.05114901]
 [1.         0.72349531 0.523

(array([3.12758003e+50, 2.01324590e+50, 1.48962555e+50, 1.16537247e+50]),
 2.345,
 4.567)

In [352]:


##############

# How to do the function math!!!
X_data = numpy.array([1,2,3,4])
deg = 4
coeff = numpy.array([0.5,0.5,0.5,0.5,0.5])

polynomial = numpy.poly1d(numpy.ones(deg+1))
print(polynomial)
y_values = polynomial(X_data)  # Evaluate the polynomial at the dataset points
print ("Y_vals", y_values)

y_pred_poly = numpy.polyval(coeff, X_data)
print("Polyval evaluation:", y_pred_poly)

y = numpy.array([6,33,124,346])

print("Predicted, y - y_pred_poly", y - y_pred_poly)
print("MSE", getMSEOld(y_pred_poly, y))

# Try the other guys way, ensure same loss
X_vec = numpy.ones((1,len(X_data)))
for i in range(0,deg):
    X_vec = numpy.vstack((X_vec, X_data**(i+1)))
X_vec = X_vec.T
print("X Vector after transpose:", X_vec)
print("Predicted, y - x.dot(coeff)", y - X_vec.dot(coeff))
print("MSE_test", getMSEOld(X_vec.dot(coeff), y))

# Try a 3rd time using this Vander Matrix
matrix_vander = numpy.vander(X_data, deg+1, True) # bool flag flips if it's leading
print("The vander matrix:\n", matrix_vander)

print("Predicted Y - Vander", y - matrix_vander.dot(coeff))
print("MSE Vander", getMSEOld(matrix_vander.dot(coeff), y))



   4     3     2
1 x + 1 x + 1 x + 1 x + 1
Y_vals [  5.  31. 121. 341.]
Polyval evaluation: [  2.5  15.5  60.5 170.5]
Predicted, y - y_pred_poly [  3.5  17.5  63.5 175.5]
MSE 8787.75
X Vector after transpose: [[  1.   1.   1.   1.   1.]
 [  1.   2.   4.   8.  16.]
 [  1.   3.   9.  27.  81.]
 [  1.   4.  16.  64. 256.]]
Predicted, y - x.dot(coeff) [  3.5  17.5  63.5 175.5]
MSE_test 8787.75
The vander matrix:
 [[  1   1   1   1   1]
 [  1   2   4   8  16]
 [  1   3   9  27  81]
 [  1   4  16  64 256]]
Predicted Y - Vander [  3.5  17.5  63.5 175.5]
MSE Vander 8787.75


In [353]:
import numpy as np

def GD_Test(X,y,n,weights):
    itr = 2000
    lrate = 0.001
    Ein_list = np.zeros(itr)
    Ein=0
    for i in range(itr):
        weights -= lrate * (2/n) * (((X.dot(weights) - y).T).dot(X).T)
        Ein_list[i] = getMSEOld(X.dot(weights), y)
    Ein = Ein_list[itr-1]
    return weights,Ein

def fitData_Test(data,d):
    y = data[:,1].reshape((len(data),1))
    x = data[:,0]
    print("X data, pre mod:\n", x)
    X = np.ones((1,len(data)))
    for i in range(d):
        X = np.vstack((X, x**(i+1)))
    X= X.T
    print("X data after transpose:\n", X)  # vector of X, evaluated. Nxd+1
    #print("Y data:\n", y)
    
    #initialize weights
    weights = np.random.random([d+1,1])
    print("Weights:", weights)
    print("Y_pred", X.dot(weights))
    #print(((X.dot(weights) - y).T).dot(X).T)
    n=len(data)

    #function setup
    weights,Ein = GD_Test(X,y,n,weights)
    return weights,Ein


def getMSE_TEst(y_pred,y):
    mse = np.mean((y-y_pred)**2)
    return mse


N = 3        # Size of the training dataset
variance = 0.003  # Variance, sigma^2
degree_d=6      # Degree of the model polynomial


data_set = getData(N, variance)
weights2,Ein = fitData_Test(data_set, degree_d)

X data, pre mod:
 [0.09381611 0.99632864 0.81843407]
X data after transpose:
 [[1.00000000e+00 9.38161124e-02 8.80146294e-03 8.25719036e-04
  7.74657499e-05 7.26753549e-06 6.81811926e-07]
 [1.00000000e+00 9.96328643e-01 9.92670764e-01 9.89026315e-01
  9.85395246e-01 9.81777508e-01 9.78173052e-01]
 [1.00000000e+00 8.18434067e-01 6.69834321e-01 5.48215227e-01
  4.48678018e-01 3.67213375e-01 3.00539936e-01]]
Weights: [[0.48885599]
 [0.19014805]
 [0.15018242]
 [0.00680995]
 [0.34112427]
 [0.86175324]
 [0.95621077]]
Y_pred [[0.50805573]
 [2.95165465]
 [1.50569209]]
